# Import

In [1]:
import sys
sys.path.append('../backend/')

import math
import random
import pandas as pd
from event import Event
from user_agents import parse
from bs4 import BeautifulSoup
import requests
from lxml import etree
import xmltodict

In [2]:
# !pip install xmltodict

# Test reading products list

In [3]:
url = 'https://www.premierdeadsea-usa.com/feeds/google_shopping_feed_usa.xml'

In [4]:
response = requests.get(url)

In [5]:
root = etree.fromstring(response.content)

In [6]:
root

<Element rss at 0x7fe32e179480>

In [7]:
dict_data = xmltodict.parse(response.content)

In [8]:
dict_data

OrderedDict([('rss',
              OrderedDict([('@xmlns:g', 'http://base.google.com/ns/1.0'),
                           ('@version', '2.0'),
                           ('channel',
                            OrderedDict([('title', 'Name of data feed'),
                                         ('link', 'http://www.example.com'),
                                         ('description',
                                          'Description of data feed'),
                                         ('item',
                                          [OrderedDict([('g:id', 'A16e'),
                                                        ('title',
                                                         'Classic Day Cream Complex'),
                                                        ('description',
                                                         "A multi-functional day cream moisturizer that preserves skin and its natural, precious balance. As the cream treats visible signs of 

In [9]:
dict_data.keys()

odict_keys(['rss'])

In [10]:
dict_data['rss']['channel'].keys()

odict_keys(['title', 'link', 'description', 'item'])

In [11]:
df = pd.DataFrame.from_dict(dict_data['rss']['channel'])

In [12]:
df.head()

,title,link,description,item
0,Name of data feed,http://www.example.com,Description of data feed,"{'g:id': 'A16e', 'title': 'Classic Day Cream C..."
1,Name of data feed,http://www.example.com,Description of data feed,"{'g:id': 'K30n', 'title': 'Biox Ultra Thermal ..."
2,Name of data feed,http://www.example.com,Description of data feed,"{'g:id': 'K31n', 'title': 'Biox Ultra Intensiv..."
3,Name of data feed,http://www.example.com,Description of data feed,"{'g:id': 'B101', 'title': 'Beautifying Luxury ..."
4,Name of data feed,http://www.example.com,Description of data feed,"{'g:id': 'B101b', 'title': 'Beautifying Luxury..."


In [13]:
len(df)

395

In [33]:
s = df['item']

In [34]:
s.head()

0    {'g:id': 'A16e', 'title': 'Classic Day Cream C...
1    {'g:id': 'K30n', 'title': 'Biox Ultra Thermal ...
2    {'g:id': 'K31n', 'title': 'Biox Ultra Intensiv...
3    {'g:id': 'B101', 'title': 'Beautifying Luxury ...
4    {'g:id': 'B101b', 'title': 'Beautifying Luxury...
Name: item, dtype: object

In [40]:
s[0].keys()

odict_keys(['g:id', 'title', 'description', 'g:google_product_category', 'g:product_type', 'link', 'g:image_link', 'g:additional_image_link', 'g:condition', 'g:availability', 'g:price', 'g:brand', 'g:gtin', 'g:mpn', 'g:identifier_exists', 'g:gender', 'g:age_group', 'g:size', 'g:item_group_id', 'g:color', 'g:material', 'g:pattern', 'g:tax', 'g:shipping', 'g:multipack', 'g:adult', 'g:adwords_grouping', 'g:adwords_labels', 'adwords_redirect', 'g:unit_pricing_measure', 'g:unit_pricing_base_measure', 'g:energy_efficiency_class', 'g:online_only'])

In [41]:
s[0]

OrderedDict([('g:id', 'A16e'),
             ('title', 'Classic Day Cream Complex'),
             ('description',
              "A multi-functional day cream moisturizer that preserves skin and its natural, precious balance. As the cream treats visible signs of aging, it builds skin's resistance to the cold, wind, and hard climates. As well it helps blocks harmful UVA and UVB rays from ever damaging the skin. These protective measures keep skin soft and smooth while preventing wrinkles and fine lines from ever forming in the first place. Suited for normal to dry skin and when used daily, skin looks supple, even, and young again.A nourishing midweight cream yet non-greasy delivering hydration and protection to all skin types, including sensitive skin and absorbs without leaving a sticky film, for protected, nourished, younger-looking skin.HYPOALLERGENIC AND SENSITIVE SKIN TESTED - Advanced and gentle cream to treat wrinkles, features a non-comedogenic formula.DERMATOLOGIST-TESTED - Non-g

In [39]:
s[0]['g:id']

'A16e'

In [51]:
df['id'] = df['item'].apply(lambda x: x['g:id'])
df['title'] = df['item'].apply(lambda x: x['title'])
df['link'] = df['item'].apply(lambda x: x['link'])
df['image_link'] = df['item'].apply(lambda x: x['g:image_link'])
df['price'] = df['item'].apply(lambda x: x['g:price'])

In [52]:
df = df[['id', 'title', 'price', 'link', 'image_link']]

In [53]:
df.head()

,id,title,price,link,image_link
0,A16e,Classic Day Cream Complex,109.99 USD,https://www.premierdeadsea-usa.com/en/day-crea...,https://www.premierdeadsea-usa.com/pub/media/c...
1,K30n,Biox Ultra Thermal Beauty Experience Mask,699.99 USD,https://www.premierdeadsea-usa.com/en/biox-ult...,https://www.premierdeadsea-usa.com/pub/media/c...
2,K31n,Biox Ultra Intensive Age - Defying Cream - Age...,599.99 USD,https://www.premierdeadsea-usa.com/en/biox-ult...,https://www.premierdeadsea-usa.com/pub/media/c...
3,B101,Beautifying Luxury Nail Kit - Milk & Honey,59.99 USD,https://www.premierdeadsea-usa.com/en/beutifyi...,https://www.premierdeadsea-usa.com/pub/media/c...
4,B101b,Beautifying Luxury Nail Kit - Passion Fruit,59.99 USD,https://www.premierdeadsea-usa.com/en/beautify...,https://www.premierdeadsea-usa.com/pub/media/c...


In [54]:
len(df)

395

In [56]:
df.to_csv('premier_products.csv', index=False)

# Test reading and chossing best seller product

In [59]:
df_best = pd.read_excel('../clients/premier_staging/data/Best Sellers-LIVE.xlsx')
df_best.head()

,id
0,A82e
1,A83e
2,B232
3,B233
4,K17


In [60]:
len(df_best)

53

In [74]:
prod = df_best.sample(1)['id'].values[0]
prod

'PS21'

In [75]:
df[df.id == prod]

,id,title,price,link,image_link
244,PS21,Supreme Brightening And Pore Clarifying Cleans...,99.99 USD,https://www.premierdeadsea-usa.com/en/supreme-...,https://www.premierdeadsea-usa.com/pub/media/c...


In [79]:
df[df.id == 'prod']['title'].values

array([], dtype=object)

In [77]:
df[df.id == prod]['title'].values[0]

'Supreme Brightening And Pore Clarifying Cleansing Foam'

# Test reading CTA

In [81]:
df_cta = pd.read_csv('../clients/premier_staging/data/cta-general.csv')
len(df_cta)

61

In [82]:
df_cta.head()

,id,cta
0,911,WATCH NOW
1,912,EXPLORE NOW
2,913,LEARN MORE
3,914,FIND OUT MORE
4,915,HOW IT WORKS


In [89]:
df_cta[df_cta.id == 914]['cta'].values[0]

'FIND OUT MORE'

# Test reading 4 row copy

In [92]:
df_copy4 = pd.read_excel('../clients/premier_staging/data/Premier Copy 4 Rows Mobile -Live.xlsx')
df_copy4.head()

,id,row1,row2,row3,row4,name,prod_id
0,1,Love this product.,Goes on very nicely.,I found the smell very natural,and subtle,Kelley,A82e
1,2,An ultimate winner for winter!,love the scent,and how it makes my skin feel.,NaN,Ellis,A82e
2,3,With just one use,I noticed dramatic results!,The most nourishing body butter I,have ever used! Worth every penny!,Lori,A83e
3,4,Got this for Christmas,and I am,"very impressed,",this stuff is amazing!,Renee',A83e
4,5,This product beat anything,"the doctors gave me to use,",even by perscription. My skin was so dry,it flaked and looked horrible...,Barbara,A83e


In [95]:
df_copy4[df_copy4.prod_id == 'A83e']

,id,row1,row2,row3,row4,name,prod_id
2,3,With just one use,I noticed dramatic results!,The most nourishing body butter I,have ever used! Worth every penny!,Lori,A83e
3,4,Got this for Christmas,and I am,"very impressed,",this stuff is amazing!,Renee',A83e
4,5,This product beat anything,"the doctors gave me to use,",even by perscription. My skin was so dry,it flaked and looked horrible...,Barbara,A83e


In [98]:
df_copy4[df_copy4.prod_id == prod]

,id,row1,row2,row3,row4,name,prod_id
89,90,"I can now walk comfortably,",even without makeup.,I am glad I came across,this product.,Sasha,PS21
90,91,My husband and I both love,this cleanser because,it makes your skin feel,smooth almost instantly.,Jennifer B,PS21


In [100]:
copy = df_copy4[df_copy4.prod_id == prod].sample(1)
copy

,id,row1,row2,row3,row4,name,prod_id
89,90,"I can now walk comfortably,",even without makeup.,I am glad I came across,this product.,Sasha,PS21


In [101]:
row1 = copy['row1'].values[0]
row2 = copy['row2'].values[0]
row3 = copy['row3'].values[0]
row4 = copy['row4'].values[0]
name = copy['name'].values[0]
row1, row2, row3, row4, name

('I can now walk comfortably, ',
 'even without makeup. ',
 'I am glad I came across',
 'this product.',
 'Sasha')

In [103]:
'row5' in copy

False

# Test reading 2 row copy

In [97]:
df_copy2 = pd.read_excel('../clients/premier_staging/data/Premier Copy 4 Rows Mobile -Live.xlsx')
df_copy2.head()

,id,row1,row2,row3,row4,name,prod_id
0,1,Love this product.,Goes on very nicely.,I found the smell very natural,and subtle,Kelley,A82e
1,2,An ultimate winner for winter!,love the scent,and how it makes my skin feel.,NaN,Ellis,A82e
2,3,With just one use,I noticed dramatic results!,The most nourishing body butter I,have ever used! Worth every penny!,Lori,A83e
3,4,Got this for Christmas,and I am,"very impressed,",this stuff is amazing!,Renee',A83e
4,5,This product beat anything,"the doctors gave me to use,",even by perscription. My skin was so dry,it flaked and looked horrible...,Barbara,A83e


# Test reading behavior_mapping 

In [ ]:
df = pd.read_excel('../data/behavior_mapping.xlsx')
df.head()

In [ ]:
df.columns

In [ ]:
s = '711,712,713,714,213,311,312'
s.split(',')

In [ ]:
def expand_commas(x):
    return x.split(',')

In [ ]:
df['copy'] = df['copy'].apply(expand_commas)
df['cta'] = df['cta'].apply(expand_commas)

In [ ]:
df

In [ ]:
df.set_index('behavior', inplace=True)
df

In [ ]:
behavior = 'BH'

In [ ]:
df.loc[behavior]

In [ ]:
copy = random.choice(df.loc[behavior]['copy'])
cta = random.choice(df.loc[behavior]['cta'])
copy, cta

## test reading copy & cta data

In [ ]:
df_copy = pd.read_csv('../data/copy-general-livia.csv')
df_copy.head()

In [ ]:
copy = 112

In [ ]:
df_copy[df_copy.id == int(copy)]

In [ ]:
res = df_copy[df_copy.id == int(copy)].iloc[0]['copy2']
res

In [ ]:
math.isnan(res)

In [ ]:
df_copy[df_copy.id == int(copy)].iloc[0]['ref']

In [ ]:
df_cta = pd.read_csv('../data/cta-general.csv')
df_cta.head()

In [ ]:
df_cta[df_cta.id == int(cta)].iloc[0]['cta']

# Test using beutiful soup to replace text in banners

In [ ]:
with open('../banners/desktop_1000x100_promotional.html', 'rt') as file:
    html = file.read()
print(html)

In [ ]:
soup = BeautifulSoup(html)
print(soup.prettify())

In [ ]:
result = soup.find(id='SEE_PLANS_AND_PRICING')
# a = HTMLDocument.find(id='SEE_PLANS_AND_PRICING')
print(result)

In [ ]:
new = f'<div id="SEE_PLANS_AND_PRICING"><span>something_else</span></div>'
new_soup = BeautifulSoup(new)
print(new_soup)

In [ ]:
result.replace_with(new_soup)

In [ ]:
print(soup.prettify())

In [ ]:
type(soup.prettify())

# Test parsing events

In [ ]:
start_msg = {
  "messageId": "c79b105b-bf2d-4b22-947e-1bfdbfac57b4",
  "receiptHandle": "AQEBYcREbIu6ZqeATmhhMiTvoWXrHfRjsgcidDeJun0kBooQZsNqxRzogm3KH+mQXwn5XYhzYpBW+3gf2Rxk/R2W2GfMnmAQcsYa2805agQO9+9bT7WQdh6WURhE9FdPTjPCp7sFZkjvupr3+7dpAbvRV/UhKSquZVH7vmRcQqXyZostB42dcSjQDRNhaoCZ/r/uSSEDTpZtCaZz2WXo8c3u8Dc6tanCtPrRzfSEYvS2ied4fviGMjy2RPC9pXSfi+ViSLydQlfuXgwJUEiFDO5Kxg==",
  "body": "{\"ip\":\"185.175.34.202\",\"agent\":\"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36\",\"screen_size\":\"2560x1440\",\"time_zone\":\"+3\"}",
  "attributes": {
    "ApproximateReceiveCount": "1",
    "SentTimestamp": "1628343373693",
    "SequenceNumber": "18863599977374960128",
    "MessageGroupId": "97e53c55-cdc0-40b4-87a0-a51cdd4c17d3",
    "SenderId": "AIDAWTW4GHYIPYQUGRWYU",
    "MessageDeduplicationId": "ace111fd2d7a1ca393732763a5aec06f12cbb0df34228b4e5c3b6effe812f1d6",
    "ApproximateFirstReceiveTimestamp": "1628343373693"
  },
  "messageAttributes": {
    "Type": {
      "stringValue": "start",
      "stringListValues": [],
      "binaryListValues": [],
      "dataType": "String"
    },
    "Time": {
      "stringValue": "0",
      "stringListValues": [],
      "binaryListValues": [],
      "dataType": "Number"
    },
    "Client": {
      "stringValue": "245678",
      "stringListValues": [],
      "binaryListValues": [],
      "dataType": "String"
    },
    "UUID": {
      "stringValue": "97e53c55-cdc0-40b4-87a0-a51cdd4c17d3",
      "stringListValues": [],
      "binaryListValues": [],
      "dataType": "String"
    },
    "Compressed": {
      "stringValue": "false",
      "stringListValues": [],
      "binaryListValues": [],
      "dataType": "String"
    }
  },
  "md5OfBody": "5d1b50ff7bff6ab229285b4f2a22dd57",
  "md5OfMessageAttributes": "2e2edb97370053835f75a9c0bbffb8ae",
  "eventSource": "aws:sqs",
  "eventSourceARN": "arn:aws:sqs:us-east-1:454654574096:events.fifo",
  "awsRegion": "us-east-1"
}


In [ ]:
event_obj = Event()
event_obj.parse(start_msg)

In [ ]:
print(event_obj)

In [ ]:
ua_string = event_obj.body['agent']
user_agent = parse(ua_string)

In [ ]:
user_agent.browser.family, user_agent.browser.version

In [ ]:
user_agent.os.family, user_agent.os.version

In [ ]:
user_agent.device.family, user_agent.device.brand, user_agent.device.model

In [ ]:
user_agent.is_mobile, user_agent.is_tablet, user_agent.is_touch_capable, user_agent.is_pc, user_agent.is_bot